In [2]:
import csv

def load_data(filename):
    with open(filename, 'r', newline='') as f:
        reader = csv.reader(f)
        return [tuple(reversed(tuple(map(int, row)))) for row in reader]

file_name = 'day9.txt'
coordinates = load_data(file_name)

In [1]:
1+1

2

In [3]:
import numpy as np
rect_sizes = np.zeros((len(coordinates), len(coordinates)))

for i in range(len(coordinates)):
    for j in range(len(coordinates)):
        side_a = abs(coordinates[j][0] - coordinates[i][0]) + 1
        side_b = abs(coordinates[j][1] - coordinates[i][1]) + 1
        rect_sizes[i,j] = side_a * side_b


In [4]:
rect_sizes

array([[1.00000e+00, 1.22100e+03, 1.39194e+05, ..., 4.92075e+05,
        1.21500e+05, 1.00000e+02],
       [1.22100e+03, 1.00000e+00, 1.14000e+02, ..., 9.86175e+05,
        2.43500e+05, 1.22100e+05],
       [1.39194e+05, 1.14000e+02, 1.00000e+00, ..., 1.26133e+06,
        5.18655e+05, 2.60073e+05],
       ...,
       [4.92075e+05, 9.86175e+05, 1.26133e+06, ..., 1.00000e+00,
        3.06000e+02, 3.71790e+05],
       [1.21500e+05, 2.43500e+05, 5.18655e+05, ..., 3.06000e+02,
        1.00000e+00, 1.21500e+03],
       [1.00000e+02, 1.22100e+05, 2.60073e+05, ..., 3.71790e+05,
        1.21500e+03, 1.00000e+00]], shape=(496, 496))

In [5]:
np.max(rect_sizes)

np.float64(4715966250.0)

In [6]:
max_x = np.max(np.array([x[0] for x in coordinates]))
print(max_x)
max_y = np.max(np.array([x[1] for x in coordinates]))
print(max_y)

98199
98273


In [14]:
import numpy as np
import scipy.sparse
#layout = np.full((max_x+1, max_y +1), '.', dtype='U1')
layout = scipy.sparse.lil_matrix((max_x + 1, max_y + 1), dtype=np.int8)
R_val = 1
G_val = 2

In [15]:
def color_in_between(coordinate1, coordinate2, layout):

    r1, c1 = coordinate1
    r2, c2 = coordinate2

    if r1 == r2:
        c_start, c_end = sorted([c1, c2])
        layout[r1, c_start + 1 : c_end] = G_val
    elif c1 == c2:
        r_start, r_end = sorted([r1, r2])
        layout[r_start + 1 : r_end, c1] = G_val


for i, coordinate in enumerate(coordinates):
    layout[coordinate] = R_val

    if i == 0:
        prev_coordinate = coordinates[-1]
        next_coordinate = coordinates[i+1]
    elif i == len(coordinates)-1:
        prev_coordinate = coordinates[i-1]
        next_coordinate = coordinates[1]
    else:
        prev_coordinate = coordinates[i-1]
        next_coordinate = coordinates[i+1]
    

    color_in_between(coordinate, next_coordinate, layout)
    color_in_between(coordinate, prev_coordinate, layout)




In [19]:
from scipy.ndimage import binary_fill_holes
layout_csr = layout.tocsr()
mask_R = (layout_csr == R_val) 
mask_G = (layout_csr == G_val) 
outline_mask_sparse = mask_R + mask_G
dense_outline_mask = outline_mask_sparse.toarray().astype(bool) 

filled_mask = binary_fill_holes(dense_outline_mask)

MemoryError: Unable to allocate 8.99 GiB for an array with shape (98200, 98274) and data type int8

In [ ]:
spots_to_fill_G = (filled_mask == True) & (dense_outline_mask == False) # Identify the new holes

# Apply these changes back to the sparse matrix (convert back to LIL temporarily if needed)
# A more direct way to apply changes from a dense boolean mask:
r_fill, c_fill = np.where(spots_to_fill_G)
layout[r_fill, c_fill] = G_val 

In [ ]:
dense_outline_mask = outline_mask_sparse.toarray().astype(bool) 

filled_mask = binary_fill_holes(dense_outline_mask)

# Find spots that were empty (0) but are now filled (True in the mask)
# Compare the dense 'filled' mask with the original dense 'outline' mask
spots_to_fill_G = (filled_mask == True) & (dense_outline_mask == False) # Identify the new holes

# Apply these changes back to the sparse matrix (convert back to LIL temporarily if needed)
# A more direct way to apply changes from a dense boolean mask:
r_fill, c_fill = np.where(spots_to_fill_G)
layout_sparse[r_fill, c_fill] = G_val 

In [ ]:
from scipy.ndimage import binary_fill_holes
outline_mask = (layout == 'R') | (layout == 'G')
filled_mask = binary_fill_holes(outline_mask)
spots_to_fill_G = (filled_mask == True) & (layout == '.')
layout[spots_to_fill_G] = "G"
layout

In [78]:
rect_sizes

array([[ 1.,  5., 35., 21., 15., 30., 18.,  3.],
       [ 5.,  1.,  7., 21., 15., 50., 30., 15.],
       [35.,  7.,  1.,  3.,  9., 30., 50., 25.],
       [21., 21.,  3.,  1.,  3., 24., 40., 15.],
       [15., 15.,  9.,  3.,  1.,  8., 24.,  9.],
       [30., 50., 30., 24.,  8.,  1.,  3., 18.],
       [18., 30., 50., 40., 24.,  3.,  1.,  6.],
       [ 3., 15., 25., 15.,  9., 18.,  6.,  1.]])

In [79]:

row_indices, col_indices = np.tril_indices(rect_sizes.shape[0], k=-1)

# 3. Extract the actual values using these indices
lower_triangle_values = rect_sizes[row_indices, col_indices]
sort_indices_ascending = np.argsort(-lower_triangle_values)

sorted_values_desc = lower_triangle_values[sort_indices_ascending]
sorted_rows_desc = row_indices[sort_indices_ascending]
sorted_cols_desc = col_indices[sort_indices_ascending]

values_list = sorted_values_desc.tolist()

# List of indices (as tuples)
indices_list = list(zip(sorted_rows_desc, sorted_cols_desc))
print(f"Sorted Values (Descending): {values_list}\n")
print(f"Corresponding Indices (Descending by value): {indices_list}")


Sorted Values (Descending): [50.0, 50.0, 40.0, 35.0, 30.0, 30.0, 30.0, 25.0, 24.0, 24.0, 21.0, 21.0, 18.0, 18.0, 15.0, 15.0, 15.0, 15.0, 9.0, 9.0, 8.0, 7.0, 6.0, 5.0, 3.0, 3.0, 3.0, 3.0]

Corresponding Indices (Descending by value): [(np.int64(5), np.int64(1)), (np.int64(6), np.int64(2)), (np.int64(6), np.int64(3)), (np.int64(2), np.int64(0)), (np.int64(5), np.int64(0)), (np.int64(6), np.int64(1)), (np.int64(5), np.int64(2)), (np.int64(7), np.int64(2)), (np.int64(5), np.int64(3)), (np.int64(6), np.int64(4)), (np.int64(3), np.int64(1)), (np.int64(3), np.int64(0)), (np.int64(7), np.int64(5)), (np.int64(6), np.int64(0)), (np.int64(4), np.int64(0)), (np.int64(4), np.int64(1)), (np.int64(7), np.int64(3)), (np.int64(7), np.int64(1)), (np.int64(7), np.int64(4)), (np.int64(4), np.int64(2)), (np.int64(5), np.int64(4)), (np.int64(2), np.int64(1)), (np.int64(7), np.int64(6)), (np.int64(1), np.int64(0)), (np.int64(4), np.int64(3)), (np.int64(3), np.int64(2)), (np.int64(6), np.int64(5)), (np.int64(

In [45]:
rect_sizes

array([[ 1.,  5., 35., 21., 15., 30., 18.,  3.],
       [ 5.,  1.,  7., 21., 15., 50., 30., 15.],
       [35.,  7.,  1.,  3.,  9., 30., 50., 25.],
       [21., 21.,  3.,  1.,  3., 24., 40., 15.],
       [15., 15.,  9.,  3.,  1.,  8., 24.,  9.],
       [30., 50., 30., 24.,  8.,  1.,  3., 18.],
       [18., 30., 50., 40., 24.,  3.,  1.,  6.],
       [ 3., 15., 25., 15.,  9., 18.,  6.,  1.]])

In [82]:
def are_rectangle_values_valid(arr, coord1, coord2, valid_values):
    # Ensure coordinates define the correct slice regardless of order
    x_1, y_1 = coord1
    x_2, y_2 = coord2

    r_start = min(coord1)
    r_end = max(coord1)
    c_start = min(coord2)
    c_end = max(coord2)

    r_start = min(x_1, x_2)
    r_end = max(x_1, x_2)
    c_start = min(y_1, y_2)
    c_end = max(y_1, y_2)

    # Slice the specific rectangle (inclusive bounds using +1)
    rectangle_slice = arr[r_start : r_end + 1, c_start : c_end + 1]
    return np.isin(rectangle_slice, valid_values).all()

i = 0
while True:
    x,y = indices_list[i]
    coord1 = coordinates[x]
    coord2 = coordinates[y]

    if are_rectangle_values_valid(layout, coord1,coord2, ["R", "G"]):
        print(values_list[i])
        break

    i += 1

24.0


In [38]:
layout

array([['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', 'R', 'G', 'G', 'G', 'R', '.'],
       ['.', '.', '.', '.', '.', '.', '.', 'G', 'G', 'G', 'G', 'G', '.'],
       ['.', '.', 'R', 'G', 'G', 'G', 'G', 'R', 'G', 'G', 'G', 'G', '.'],
       ['.', '.', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', '.'],
       ['.', '.', 'R', 'G', 'G', 'G', 'G', 'G', 'G', 'R', 'G', 'G', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', 'G', 'G', 'G', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', 'R', 'G', 'R', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']],
      dtype='<U1')